In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir('/content/gdrive/My Drive/AGGCN-master/semeval')


**Entity Extraction**

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ghef8_rp
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ghef8_rp
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=5713281b8669d9e4c8d61b314dd40865d34aec3ba5c63219b1113b01a2abb5f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-qsrirebe/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
# !pip install sklearn_crfsuite
!pip install seqeval

In [0]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional, Dropout
from keras.models import Model, Input
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

# from sklearn_crfsuite.metrics import flat_classification_report
# from sklearn.metrics import f1_score
# from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from keras.preprocessing.text import text_to_word_sequence
import pickle

In [0]:
#Reading the csv file
import pandas as pd
df = pd.read_csv('dataset/semeval/ner_dataset_semeval1.csv', encoding = "ISO-8859-1")

In [0]:
df.head()

,Unnamed: 0,Sentence #,tok,pos,tag
0,0,0.0,The,DT,O
1,1,NaN,system,NN,O
2,2,NaN,as,IN,O
3,3,NaN,described,VBD,O
4,4,NaN,above,RB,O


In [0]:
df.drop(['Unnamed: 0'],axis = 1)

,Sentence #,tok,pos,tag
0,0.0,The,DT,O
1,NaN,system,NN,O
2,NaN,as,IN,O
3,NaN,described,VBD,O
4,NaN,above,RB,O
...,...,...,...,...
152624,NaN,to,TO,O
152625,NaN,expose,VB,O
152626,NaN,the,DT,O
152627,NaN,nerve,NN,O


In [0]:
df.describe()

,Unnamed: 0,Sentence #
count,152629.000000,8000.00000
mean,76314.000000,3999.50000
std,44060.341455,2309.54541
min,0.000000,0.00000
25%,38157.000000,1999.75000
50%,76314.000000,3999.50000
75%,114471.000000,5999.25000
max,152628.000000,7999.00000


In [0]:
df.isnull().sum()

Unnamed: 0         0
Sentence #    144629
tok                1
pos                0
tag                0
dtype: int64

In [0]:
df = df.fillna(method = 'ffill')

In [0]:
df.isnull().sum()

Unnamed: 0    0
Sentence #    0
tok           0
pos           0
tag           0
dtype: int64

In [0]:
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['tok'].values.tolist(),
                                                       s['pos'].values.tolist(),
                                                       s['tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped[self.n_sent]
            self.n_sent +=1
            return s
        except:
            return None

In [0]:
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'The system as described above has its greatest application in an arrayed configuration of antenna elements .'

In [0]:
# for i in range(len(sentences)):
#   print(sentences[i])


In [0]:
df.to_csv (r'dataset/semeval/new_train_dataset_stanza.csv')

In [0]:
df_new = pd.read_csv('dataset/semeval/new_train_dataset_stanza.csv')

In [0]:
df_new.head()

,Unnamed: 0,Unnamed: 0.1,Sentence #,tok,pos,tag
0,0,0,0.0,The,DT,O
1,1,1,NaN,system,NN,O
2,2,2,NaN,as,IN,O
3,3,3,NaN,described,VBD,O
4,4,4,NaN,above,RB,O


In [0]:
df_new.drop(['Unnamed: 0','Unnamed: 0.1'],axis = 1)

,Sentence #,tok,pos,tag
0,0.0,The,DT,O
1,NaN,system,NN,O
2,NaN,as,IN,O
3,NaN,described,VBD,O
4,NaN,above,RB,O
...,...,...,...,...
152624,NaN,to,TO,O
152625,NaN,expose,VB,O
152626,NaN,the,DT,O
152627,NaN,nerve,NN,O


In [0]:
getter.grouped

Sentence #
0.0       [(The, DT, O), (most, RBS, O), (common, JJ, O)...
1.0       [(The, DT, O), (company, NN, B-SUBJ), (fabrica...
2.0       [(The, DT, O), (school, NN, O), (master, NN, B...
3.0       [(The, DT, O), (suspect, NN, O), (dumped, VBD,...
4.0       [(Avian, NNP, O), (influenza, NNP, B-SUBJ), (i...
                                ...                        
2712.0    [(After, IN, O), (seating, VBG, O), (all, PDT,...
2713.0    [(The, DT, O), (minister, NN, O), (attributed,...
2714.0    [(The, DT, O), (umbrella, NN, B-SUBJ), (frame,...
2715.0    [(Manos, NNS, O), (:, :, O), (The, DT, O), (Ha...
2716.0    [(A, DT, O), (few, JJ, O), (days, NNS, O), (be...
Length: 2717, dtype: object

In [0]:
sent = getter.get_text()
print(sent)

[('The', 'DT', 'O'), ('author', 'NN', 'B-SUBJ'), ('of', 'IN', 'O'), ('a', 'DT', 'O'), ('keygen', 'NN', 'O'), ('uses', 'VBZ', 'O'), ('a', 'DT', 'O'), ('disassembler', 'NN', 'B-OBJ'), ('to', 'TO', 'O'), ('look', 'VB', 'O'), ('at', 'IN', 'O'), ('the', 'DT', 'O'), ('raw', 'JJ', 'O'), ('assembly', 'NN', 'O'), ('code', 'NN', 'O'), ('.', '.', 'O')]


In [0]:
sentences = getter.sentences

In [0]:
sentences

In [0]:
# Number of data points passed in each iteration
batch_size = 64 
# Passes through entire dataset
epochs = 100
# Maximum length of sentence
max_len = 75 
# Dimension of embedding vector
embedding = 40

In [0]:
#Getting unique words and labels from data
words = list(df['tok'].unique())
tags = list(df['tag'].unique())
# Dictionary word:index pair
# word is key and its value is corresponding index
word_to_index = {w : i + 2 for i, w in enumerate(words)}
word_to_index["UNK"] = 1
word_to_index["PAD"] = 0

# Dictionary lable:index pair
# label is key and value is index.
tag_to_index = {t : i + 1 for i, t in enumerate(tags)}
tag_to_index["PAD"] = 0

idx2word = {i: w for w, i in word_to_index.items()}
idx2tag = {i: w for w, i in tag_to_index.items()}

In [0]:
word_to_index1 = {w : i + 2 for i, w in enumerate(text)}
word_to_index1["UNK"] = 1
word_to_index1["PAD"] = 0

In [0]:
print("The word herbicide is identified by the index: {}".format(word_to_index["herbicide"]))
print("The label B-SUBJ is identified by the index: {}".format(tag_to_index["B-SUBJ"]))

The word herbicide is identified by the index: 21611
The label B-SUBJ is identified by the index: 2


In [0]:
# Converting each sentence into list of index from list of tokens
X = [[word_to_index[w[0]] for w in s] for s in sentences]

# Padding each sequence to have same length  of each word
X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])

In [0]:
# Convert label to index
y_tagged = [[tag_to_index[w[2]] for w in s] for s in sentences]

# padding
y = pad_sequences(maxlen = max_len, sequences = y_tagged, padding = "post", value = tag_to_index["PAD"])

In [0]:
num_tag = df['tag'].nunique()
# One hot encoded labels
tag_labels = {tag:i for i,tag in enumerate(df['tag'].unique())}
y = [to_categorical(i, num_classes = num_tag + 1) for i in y]
# y = [[tag_labels[t] for t in i] for i in y]

In [0]:
num_tag

5

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10)


In [0]:
print("Size of training input data : ", X_train.shape)
print("Size of training output data : ", np.array(y_train).shape)
print("Size of testing input data : ", X_test.shape)
print("Size of testing output data : ", np.array(y_test).shape)

Size of training input data :  (7200, 75)
Size of training output data :  (7200, 75, 6)
Size of testing input data :  (800, 75)
Size of testing output data :  (800, 75, 6)


In [0]:
# print(w[0] for w in text[0])

<generator object <genexpr> at 0x7f41a306e3b8>


In [0]:
# Let's check the first sentence before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[0] for w in sentences[1]]))
print('*****After Processing first sentence : *****\n ', X[1])

*****Before Processing first sentence : *****
 The child was carefully wrapped and bound into the cradle by means of a cord .
*****After Processing first sentence : *****
  [ 2 19 20 21 22 23 24 25 26 27 28 29 15 30 31 18  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0]


In [0]:
# First label before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[2] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', y[0])

Bidirectional LSTM-CRF Network

In [0]:
num_tags = df['tag'].nunique()
# Model architecture
input = Input(shape = (max_len,))
model = Embedding(input_dim = len(words) + 2, output_dim = embedding*2, input_length = max_len)(input) #(+2 for PAD and UNK)
model = Bidirectional(LSTM(units = 50, return_sequences=True, recurrent_dropout=0.49, recurrent_activation='tanh'))(model)
model = Dropout(0.49)(model)
model = TimeDistributed(Dense(7, activation="relu"))(model)
crf = CRF(num_tags+1)  # CRF layerembedding, +1 for PAD
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 75)                0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 75, 80)            1729280   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 75, 100)           52400     
_________________________________________________________________
dropout_9 (Dropout)          (None, 75, 100)           0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 75, 7)             707       
_________________________________________________________________
crf_12 (CRF)                 (None, 75, 6)             96        
Total params: 1,782,483
Trainable params: 1,782,483
Non-trainable params: 0
________________________________________________

In [0]:
checkpointer = ModelCheckpoint(filepath = 'model.h5',
                       verbose = 1,   #1 for info and 0 to debug
                       mode = 'auto',   #loss-min but with auto he direction is inferred automatically based on the quantity being monitored
                       save_best_only = True,
                       monitor='val_loss')

In [0]:
history = model.fit(X_train, np.array(y_train), batch_size=batch_size, epochs=20,
                    validation_split=0.1, callbacks=[checkpointer])

Train on 6480 samples, validate on 720 samples
Epoch 1/20
6480/6480 [==============================] - 19s 3ms/step - loss: 0.7584 - crf_viterbi_accuracy: 0.6804 - val_loss: 0.1710 - val_crf_viterbi_accuracy: 0.9698

Epoch 00001: val_loss improved from inf to 0.17101, saving model to model.h5
Epoch 2/20
6480/6480 [==============================] - 17s 3ms/step - loss: 0.1312 - crf_viterbi_accuracy: 0.9716 - val_loss: 0.0868 - val_crf_viterbi_accuracy: 0.9723

Epoch 00002: val_loss improved from 0.17101 to 0.08680, saving model to model.h5
Epoch 3/20
6480/6480 [==============================] - 17s 3ms/step - loss: 0.0804 - crf_viterbi_accuracy: 0.9729 - val_loss: 0.0807 - val_crf_viterbi_accuracy: 0.9735

Epoch 00003: val_loss improved from 0.08680 to 0.08068, saving model to model.h5
Epoch 4/20
6480/6480 [==============================] - 22s 3ms/step - loss: 0.0662 - crf_viterbi_accuracy: 0.9770 - val_loss: 0.0772 - val_crf_viterbi_accuracy: 0.9757

Epoch 00004: val_loss improved fro

In [0]:
y_pred = model.predict(X_test)
from sklearn.metrics import classification_report
y_test_flat = sum([y_t.argmax(axis=1).tolist() for y_t in y_test], [])
y_pred_flat = sum([y_t.argmax(axis=1).tolist() for y_t in y_pred], [])
y_test_flat = [idx2tag[i] for i in y_test_flat]
y_pred_flat = [idx2tag[i] for i in y_pred_flat]
print(classification_report(y_test_flat, y_pred_flat))

In [0]:
i = np.random.randint(0,X_test.shape[1]) # choose a random number between 0 and len(X_te)b
p = model.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_test[i], -1)

print("Sample number {} of {} (Test Set)".format(i, X_test.shape[0]))
# Visualization
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_test[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-2], idx2tag[t], idx2tag[pred]))

Sample number 27 of 800 (Test Set)
Word           ||True ||Pred
You            : O     O
saw            : O     O
professional   : O     O
chefs          : B-SUBJ O
using          : O     O
a              : O     O
garlic         : O     B-OBJ
press          : B-OBJ B-OBJ
.              : O     O


In [0]:
# load the saved model
from keras.models import load_model
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
model= load_model('model.h5',custom_objects={'CRF':CRF, 
                                             'crf_loss':crf_loss, 
                                             'crf_viterbi_accuracy':crf_viterbi_accuracy})

In [0]:
X_test[0]

array([    2,  6530,  4470,   525,    26,  6284,    23,  9544,  1849,
          99,    26, 20572,  5734,    23, 15924,    26,  5712,    15,
          26,   115,    18,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize
# word_tokenize('Hello!')

In [0]:
text = ['The most common audits were about waste and recycling.',
        'The company fabricates plastic chairs.',
        'The school master teaches the lesson with a stick.', 
        'The system as described above has its greatest application in an arrayed configuration of antenna elements .',
        'My phone has a lot of apps']

text[3]

'The system as described above has its greatest application in an arrayed configuration of antenna elements .'

In [0]:
get_word_index = lambda x: word_to_index.get(x, word_to_index['UNK'])
final_encoded = []
for sent in text:
  final_encoded.append([get_word_index(w) for w in word_tokenize(sent)])

In [0]:
word_to_index['The']

2

In [0]:
final_encoded[2]

[2, 2554, 1105, 3498, 26, 1, 99, 30, 3098, 18]

In [0]:
X_test_custom = pad_sequences(maxlen = 75, sequences = final_encoded, padding = "post", value = word_to_index["PAD"])

In [0]:
# i = np.random.randint(0,text.shape[0])
p = model.predict(X_test_custom)
p = np.argmax(p, axis=-1)

for i, w, pred,t in zip(X_test_custom,p, text):
    # if w != 0:
    # print("{:15}: {:5} {}".format(words[w-2], idx2tag[pred]))
    print(list(i, zip(word_tokenize(t), [idx2tag[pr] for pr in pred])))

[('The', 'O'), ('most', 'O'), ('common', 'O'), ('audits', 'B-SUBJ'), ('were', 'O'), ('about', 'O'), ('waste', 'B-OBJ'), ('and', 'O'), ('recycling', 'O'), ('.', 'O')]
[('The', 'O'), ('company', 'B-SUBJ'), ('fabricates', 'O'), ('plastic', 'O'), ('chairs', 'O'), ('.', 'O')]
[('The', 'O'), ('school', 'O'), ('master', 'O'), ('teaches', 'O'), ('the', 'O'), ('lesson', 'O'), ('with', 'O'), ('a', 'O'), ('stick', 'B-OBJ'), ('.', 'O')]
[('The', 'O'), ('system', 'O'), ('as', 'O'), ('described', 'O'), ('above', 'O'), ('has', 'O'), ('its', 'O'), ('greatest', 'O'), ('application', 'O'), ('in', 'O'), ('an', 'O'), ('arrayed', 'O'), ('configuration', 'B-SUBJ'), ('of', 'O'), ('antenna', 'O'), ('elements', 'B-OBJ'), ('.', 'O')]
[('My', 'O'), ('phone', 'B-SUBJ'), ('has', 'O'), ('a', 'O'), ('lot', 'O'), ('of', 'O'), ('apps', 'B-OBJ')]


In [0]:
with open('word_to_index.pickle', 'wb') as f:
    pickle.dump(word_to_index, f)

with open('tag_to_index.pickle', 'wb') as f:
    pickle.dump(tag_to_index, f)

**AGCN**

In [0]:
!python3 prepare_vocab.py semeval/dataset/semeval semeval/dataset/vocab --glove_dir dataset/glove

loading files...
134931 tokens from 7836 examples loaded from semeval/dataset/semeval/train_new.json.
46337 tokens from 2668 examples loaded from semeval/dataset/semeval/test_new.json.
loading glove...
2195892 words loaded from glove.
building vocab...
vocab built with 17890/18218 words.
calculating oov...
train oov: 358/134931 (0.27%)
test oov: 3793/46337 (8.19%)
building embeddings...
embedding size: 17890 x 300
dumping to files...
all done.


In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [3]:
!bash train_aggcn.sh

Vocab size 17890 loaded from file
Loading data from dataset/semeval with batch size 50...
11 batches created for dataset/semeval/train_new.json
Config saved to file ./saved_models_aggcn/01/config.json
Overwriting old vocab file at ./saved_models_aggcn/01/vocab.pkl

Running with the following configs:
	data_dir : dataset/semeval
	vocab_dir : dataset/vocab
	emb_dim : 300
	pos_dim : 30
	hidden_dim : 360
	num_layers : 2
	input_dropout : 0.5
	gcn_dropout : 0.3
	cnn_dropout : 0.5
	word_dropout : 0.04
	topn : 10000000000.0
	lower : False
	heads : 3
	sublayer_first : 2
	sublayer_second : 4
	conv_l2 : 0
	pooling : max
	pooling_l2 : 0.002
	mlp_layers : 1
	no_adj : False
	rnn : True
	rnn_hidden : 300
	rnn_layers : 1
	rnn_dropout : 0.5
	lr : 0.5
	lr_decay : 0.9
	decay_epoch : 6
	optim : sgd
	num_epoch : 500
	batch_size : 50
	max_grad_norm : 5.0
	log_step : 20
	log : logs.txt
	save_epoch : 100
	save_dir : ./saved_models_aggcn
	id : 1
	info : 
	seed : 1
	cuda : True
	cpu : False
	load : False
	model

In [4]:
!python3 eval.py saved_models_aggcn/01 --dataset test_new

Loading model from saved_models_aggcn/01/best_model.pt
Finetune all embeddings.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
Vocab size 17890 loaded from file
Loading data from dataset/semeval/test_new.json with batch size 50...
54 batches created for dataset/semeval/test_new.json

Running with the following configs:
	data_dir : dataset/semeval
	vocab_dir : dataset/vocab
	emb_dim : 300
	pos_dim : 30
	hidden_dim : 360
	num_layers : 2
	input_dropout : 0.5
	gcn_dropout : 0.3
	cnn_dropout : 0.5
	word_dropout : 0.04
	topn : 10000000000.0
	lower : False
	heads : 3
	sublayer_first : 2
	sublayer_second : 4
	conv_l2 : 0
	pooling : max
	pooling_l2 : 0.002
	mlp_layers : 1
	no_adj : False
	rnn : True
	rnn_hidden : 300
	rnn_layers : 1
	rnn_dropout : 0.5
	lr : 0.5
	

Creating data in the input format
(finding POS tags)

In [0]:
!pip install stanza

     |████████████████████████████████| 194kB 8.8MB/s 


In [0]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline

2020-05-24 09:08:01 INFO: Downloading default packages for language: en (English)...
2020-05-24 09:08:34 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-05-24 09:08:34 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-05-24 09:08:34 INFO: Use device: gpu
2020-05-24 09:08:34 INFO: Loading: tokenize
2020-05-24 09:08:43 INFO: Loading: pos
2020-05-24 09:08:44 INFO: Loading: lemma
2020-05-24 09:08:44 INFO: Loading: depparse
2020-05-24 09:08:45 INFO: Loading: ner
2020-05-24 09:08:46 INFO: Done loading processors!


In [0]:
# for index, value in zip(range(len(sentences)), sentences):
#   print(value)allmo_sentences
from tqdm import tqdm
all_sentences = []
for i,val in enumerate(tqdm(sentences)):
  doc = nlp(str(val))
  all_sentences.append([(word.text, word.xpos, word.head, word.deprel) for sent in doc.sentences for word in sent.words])

Streaming output truncated to the last 5000 lines.
 38%|███▊      | 3007/8000 [04:09<08:06, 10.26it/s]

 38%|███▊      | 3009/8000 [04:09<07:44, 10.73it/s]

 38%|███▊      | 3011/8000 [04:09<07:16, 11.44it/s]

 38%|███▊      | 3013/8000 [04:10<07:07, 11.66it/s]

 38%|███▊      | 3015/8000 [04:10<06:56, 11.96it/s]

 38%|███▊      | 3017/8000 [04:10<06:58, 11.91it/s]

 38%|███▊      | 3019/8000 [04:10<06:53, 12.06it/s]

 38%|███▊      | 3021/8000 [04:10<07:26, 11.14it/s]

 38%|███▊      | 3023/8000 [04:10<07:37, 10.88it/s]

 38%|███▊      | 3025/8000 [04:11<07:12, 11.51it/s]

 38%|███▊      | 3027/8000 [04:11<07:10, 11.56it/s]

 38%|███▊      | 3029/8000 [04:11<06:58, 11.89it/s]

 38%|███▊      | 3031/8000 [04:11<06:35, 12.55it/s]

 38%|███▊      | 3033/8000 [04:11<06:48, 12.14it/s]

 38%|███▊      | 3035/8000 [04:11<06:50, 12.09it/s]

 38%|███▊      | 3037/8000 [04:12<07:06, 11.64it/s]

 38%|███▊      | 3039/8000 [04:12<07:59, 10.34it/s]

 38%|███▊      | 3041/8000 [04:12<08:05, 10.21it

In [0]:
import json
from tqdm import tqdm

with open('dataset/semeval/train.json', 'r') as f:
    distros_dict = json.load(f)
    
sentence_sub_obj = dict()
all_sentences = []
for distro in tqdm(distros_dict):
    sent_init = " ".join(distro['token'])
    doc = nlp(str(sent_init))
    final_sent = " ".join([word.text for sent in doc.sentences for word in sent.words])
    all_sentences.append([(word.text, word.xpos, word.head, word.deprel) for sent in doc.sentences for word in sent.words])
    sentence_sub_obj[final_sent] = {k:distro[k] for k in ['subj_start', 'subj_end', 'obj_start', 'obj_end', 'relation']}

100%|██████████| 8000/8000 [12:19<00:00, 10.82it/s]


In [0]:
# any([ > 1 ])nt
final_sentences = []
counts = 0
for s in all_sentences:
  head_count = list(zip(*s))[2].count(0)
  if head_count > 1:
    counts += 1
  else:
    final_sentences.append(s)
    # print(dict(zip(['token', 'stanford_pos', 'stanford_head', 'stanford_deprel'], zip(*s))))
    # input()
counts

163

In [0]:
all_sentences_dicts_initial = [dict(zip(['token', 'stanford_pos', 'stanford_head', 'stanford_deprel'], zip(*s))) for s in final_sentences]
# with open('dataset/semeval/train_new.json','w') as jsonf:
#   all_sentences_dicts_initial = json.load(jsonf)
all_sentences_dicts_final = []
skipped = 0
for i,d in enumerate(all_sentences_dicts_initial):
  df1 = {k:list(v) for k,v in d.items()}
  sent = " ".join(df1['token'])
  if sent not in sentence_sub_obj:
    # input(sent)
    skipped += 1
    continue
  df1.update(sentence_sub_obj[sent])
  df1['id'] = i
  all_sentences_dicts_final.append(df1)

skipped

In [0]:
with open('dataset/semeval/train_new.json','w') as jsonf:
  json.dump(all_sentences_dicts_final,jsonf)

In [0]:
with open('dataset/semeval/train_new.json','r') as jsonf:
  train_data_new = json.load(jsonf)

In [0]:
len(train_data_new)

512

In [0]:
pos_set = set()
dep_set = set()
for t in train_data_new:
  pos_set.update(t['stanford_pos'])
  dep_set.update(t['stanford_deprel'])

pos_dict = {k:i+2 for i,k in enumerate(pos_set)}
dep_dict = {k:i+2 for i,k in enumerate(dep_set)}

In [0]:
pos_dict

In [0]:
# set(all_sentences_dicts_final.values())

In [0]:
# any([ > 1 ])nt
final_sentences = []
counts = 0
for s in all_sentences:
  head_count = list(zip(*s))[2].count(0)
  if head_count > 1:
    counts += 1
  else:
    final_sentences.append(s)
    # print(dict(zip(['token', 'stanford_pos', 'stanford_head', 'stanford_deprel'], zip(*s))))
    # input()
counts

163

In [0]:
import stanza

doc = nlp("The most common audits were about waste and recycling.")

Trying the same using stanford core NLP


In [0]:
!wget http://nlp.stanford.edu/software/stanford-corenlp-4.0.0-models-english.jar

--2020-05-18 14:11:22--  http://nlp.stanford.edu/software/stanford-corenlp-4.0.0-models-english.jar
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-4.0.0-models-english.jar [following]
--2020-05-18 14:11:23--  https://nlp.stanford.edu/software/stanford-corenlp-4.0.0-models-english.jar
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 670717945 (640M) [application/x-java-archive]
Saving to: ‘stanford-corenlp-4.0.0-models-english.jar’

stanford-corenlp-4. 100%[===================>] 639.65M  11.0MB/s    in 59s     

2020-05-18 14:12:23 (10.8 MB/s) - ‘stanford-corenlp-4.0.0-models-english.jar’ saved [670717945/670717945]



In [0]:
!wget http://nlp.stanford.edu/software/stanford-corenlp-latest.zip

--2020-05-18 14:12:35--  http://nlp.stanford.edu/software/stanford-corenlp-latest.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/software/stanford-corenlp-latest.zip [following]
--2020-05-18 14:12:36--  https://nlp.stanford.edu/software/stanford-corenlp-latest.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 504479415 (481M) [application/zip]
Saving to: ‘stanford-corenlp-latest.zip’

stanford-corenlp-la 100%[===================>] 481.11M  8.47MB/s    in 41s     

2020-05-18 14:13:18 (11.7 MB/s) - ‘stanford-corenlp-latest.zip’ saved [504479415/504479415]



In [0]:
!unzip stanford-corenlp-latest.zip

In [0]:
!mv stanford-corenlp-4.0.0-models-english.jar stanford-corenlp-4.0.0/

In [0]:
!export CORENLP_HOME="/content/gdrive/My Drive/AGGCN-master/semeval/stanford-corenlp-4.0.0"

In [0]:
[(word.text, word.xpos, word.head, word.deprel) for sent in doc.sentences for word in sent.words]
# print(doc.entities)

[('Barack', 'NNP', 4, 'nsubj:pass'),
 ('Obama', 'NNP', 1, 'flat'),
 ('was', 'VBD', 4, 'aux:pass'),
 ('born', 'VBN', 0, 'root'),
 ('in', 'IN', 6, 'case'),
 ('Hawaii', 'NNP', 4, 'obl'),
 ('.', '.', 4, 'punct')]

In [0]:
import os
os.environ['CORENLP_HOME'] = "/content/gdrive/My Drive/AGGCN-master/semeval/stanford-corenlp-4.0.0"

In [0]:
!java --version

openjdk 11.0.7 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


The above commands when executed in terminal gave the following result.



In [0]:
Extracted the following NER entity mentions:
NLP> "The most common audits were about waste and recycling."                                                               

Sentence #1 (12 tokens):
"The most common audits were about waste and recycling."

Tokens:
[Text=" CharacterOffsetBegin=0 CharacterOffsetEnd=1 PartOfSpeech=`` Lemma=" NamedEntityTag=O]
[Text=The CharacterOffsetBegin=1 CharacterOffsetEnd=4 PartOfSpeech=DT Lemma=the NamedEntityTag=O]
[Text=most CharacterOffsetBegin=5 CharacterOffsetEnd=9 PartOfSpeech=RBS Lemma=most NamedEntityTag=O]
[Text=common CharacterOffsetBegin=10 CharacterOffsetEnd=16 PartOfSpeech=JJ Lemma=common NamedEntityTag=O]
[Text=audits CharacterOffsetBegin=17 CharacterOffsetEnd=23 PartOfSpeech=NN Lemma=audits NamedEntityTag=O]
[Text=were CharacterOffsetBegin=24 CharacterOffsetEnd=28 PartOfSpeech=VBD Lemma=be NamedEntityTag=O]
[Text=about CharacterOffsetBegin=29 CharacterOffsetEnd=34 PartOfSpeech=RB Lemma=about NamedEntityTag=O]
[Text=waste CharacterOffsetBegin=35 CharacterOffsetEnd=40 PartOfSpeech=NN Lemma=waste NamedEntityTag=O]
[Text=and CharacterOffsetBegin=41 CharacterOffsetEnd=44 PartOfSpeech=CC Lemma=and NamedEntityTag=O]
[Text=recycling CharacterOffsetBegin=45 CharacterOffsetEnd=54 PartOfSpeech=NN Lemma=recycling NamedEntityTag=O]
[Text=. CharacterOffsetBegin=54 CharacterOffsetEnd=55 PartOfSpeech=. Lemma=. NamedEntityTag=O]
[Text=" CharacterOffsetBegin=55 CharacterOffsetEnd=56 PartOfSpeech='' Lemma=" NamedEntityTag=O]

Dependency Parse (enhanced plus plus dependencies):
root(ROOT-0, waste-8)
punct(waste-8, "-1)
det(audits-5, The-2)
advmod(common-4, most-3)
amod(audits-5, common-4)
nsubj(waste-8, audits-5)
cop(waste-8, were-6)
advmod(waste-8, about-7)
cc(recycling-10, and-9)
conj:and(waste-8, recycling-10)
punct(waste-8, .-11)
punct(waste-8, "-12)
